In [1]:
import aocd
import re
import numpy as np
from math import prod
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2022, day=11)

In [2]:
monkey_regex = r".* (?P<num>\d+):\n.*: (?P<items>[0-9 ,]+)\n.*w = (?P<op>.*)\n.*by (?P<test>\d+)\n.*y (?P<iftrue>\d+)\n.*y (?P<iffalse>\d+)"

class Monkey():
    def __init__(self, def_string):
        m = re.match(monkey_regex, def_string)
        self.num = int(m.group('num'))
        self.items = list(map(int, m.group('items').split(", ")))
        self.op = m.group('op')
        self.test = int(m.group('test'))
        self.if_true = int(m.group('iftrue'))
        self.if_false = int(m.group('iffalse'))
        self.throw_count = 0
        
    def link(self, m_list):
        self.if_true = m_list[self.if_true]
        self.if_false = m_list[self.if_false]
        
    def do_op(self, old):
        if(self.op[6:] == "old"):
            match self.op[4]:
                case "+":
                    return old + old
                case "*":
                    return old * old
        else:
            match self.op[4]:
                case "+":
                    return old + int(self.op[6:])
                case "-":
                    return old - int(self.op[6:])
                case "*":
                    return old * int(self.op[6:])
                case _:
                    print("missed op")
    
    def get_item(self):
        item = self.items.pop(0)
        item = self.do_op(item)
        item = item // 3
        return item
        
    def throw(self):
        item = self.get_item()
        if(item % self.test == 0):
            self.if_true.items.append(item)
        else:
            self.if_false.items.append(item)
        self.throw_count += 1
            
    def take_turn(self):
        while(len(self.items)):
            self.throw()
        
    def __repr__(self):
        return f"Monkey {self.num}: {self.items}"
    
monkeys = list(map(Monkey, puzzle.input_data.split("\n\n")))
for m in monkeys: m.link(monkeys)

In [3]:
for i in range(20):
    for m in monkeys:
        m.take_turn()

In [4]:
throws = sorted([m.throw_count for m in monkeys])
puzzle.answer_a = throws[-1] * throws[-2]

In [5]:
class Monkey2(Monkey):
    def link(self, m_list):
        self.if_true = m_list[self.if_true]
        self.if_false = m_list[self.if_false]
        self.cap = prod([m.test for m in m_list])
        
    def get_item(self):
        item = self.items.pop(0)
        item = self.do_op(item) % self.cap
        return item

monkeys = list(map(Monkey2, puzzle.input_data.split("\n\n")))
for m in monkeys: m.link(monkeys)

In [6]:
for i in range(10000):
    for m in monkeys:
        m.take_turn()

In [7]:
throws = sorted([m.throw_count for m in monkeys])
puzzle.answer_b = throws[-1] * throws[-2]